In [ ]:
from google.colab import drive
drive.mount('/content/drive')
# drive.flush_and_unmount() # Sync data written to local VM's disk cache of Drive-mounted folder back to Google Drive
%cd /content/drive/MyDrive/PPO \baseline
!ls

In [ ]:
# %env OPENAI_API_KEY=
# !export HF_HUB_DISABLE_XET="1"; hf download Qwen/Qwen3-1.7B --local-dir models/Qwen3-1.7B
# !export HF_HUB_DISABLE_XET="1"; hf download FreedomIntelligence/medical_o1_verifier_3B --local-dir models/medical_o1_verifier_3B
!uv pip install deepspeed vllm
!uv pip install trl==0.21.0
!uv pip install flash-attn --no-build-isolation
!uv pip install "sglang[all]>=0.5.3rc0"

Using Python 3.12.11 environment at: /usr
Resolved 143 packages in 402ms
Uninstalled 2 packages in 1ms
Installed 2 packages in 3ms
 - outlines-core==0.1.26
 + outlines-core==0.2.11
 - xgrammar==0.1.24
 + xgrammar==0.1.23
Using Python 3.12.11 environment at: /usr
Audited 1 package in 103ms
Using Python 3.12.11 environment at: /usr
Audited 1 package in 111ms
Using Python 3.12.11 environment at: /usr
Resolved 164 packages in 415ms
Uninstalled 3 packages in 2ms
Installed 3 packages in 4ms
 ~ decord==0.6.0
 - outlines-core==0.2.11
 + outlines-core==0.1.26
 - xgrammar==0.1.23
 + xgrammar==0.1.24


# Qwen3-1.7B

In [ ]:
# !accelerate launch --config_file ./deepspeed_zero3.yaml \
#     --num_processes 1  \
#     --num_machines 1 \
#     --machine_rank 0 \
#     --deepspeed_multinode_launcher standard SFT_stage1.py \
#     --model_path models/Qwen3-1.7B \
#     --data_path data/medical_o1_sft.json q
#     --max_seq_len 2048 \
#     --weight_decay 0.01 \
#     --learning_rate 2e-4 \
#     --train_bsz_per_gpu 8 \
#     --output_dir ./ckpts

In [ ]:
!accelerate launch --config_file ./deepspeed_zero3.yaml \
    --num_processes 1  \
    --num_machines 1 \
    --machine_rank 0 \
    --deepspeed_multinode_launcher standard SFT_stage1.py \
    --model_path models/Qwen3-1.7B \
    --data_path data/medical_o1_sft.json q
    --max_seq_len 4096 \
    --learning_rate 5e-6 \
    --train_bsz_per_gpu 8 \
    --output_dir ./ckpts

The following values were not passed to `accelerate launch` and had defaults used instead:
	`--num_cpu_threads_per_process` was set to `6` to improve out-of-box performance when training on CPUs
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
2025-09-28 00:13:23.289160: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-09-28 00:13:23.308071: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1759018403.329394   34200 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1759018403

In [ ]:
!accelerate launch \
	--num_processes 1 \
	--num_machines 1 \
	--machine_rank 0 \
    --config_file ./deepspeed_zero3.yaml \
	--deepspeed_multinode_launcher standard RL_stage2.py \
    --model_name_or_path ./ckpts/sft_stage1/checkpoint-2-7389/tfmr \
    --reward_model_path models/medical_o1_verifier_3B \
    --value_model_path Qwen/Qwen3-0.6B \
    --dataset_name data/medical_o1_verifiable_problem.json \
    --response_length 1300 \
    --temperature 1.0 \
    --local_rollout_forward_batch_size 8 \
    --num_ppo_epochs 3 \
    --num_mini_batches 1 \
    --total_episodes 1024 \
    --per_device_train_batch_size 2 \
    --gradient_accumulation_steps 16 \
    --bf16 True \
    --output_dir ./ckpts \
    --save_strategy steps \
    --save_step 16 \
    --save_total_limit 1 \
    --eval_strategy steps \
    --eval_steps 16 \
    --kl_coef 0.03 \
    --learning_rate 5e-7 \
    --warmup_ratio 0.05 \
    --gradient_checkpointing True \
    --dataloader_num_workers 4 \
    --run_name ppo_medical_o1 \
    --num_sample_generations -1 \
    # --report_to wandb

The following values were not passed to `accelerate launch` and had defaults used instead:
	`--num_cpu_threads_per_process` was set to `6` to improve out-of-box performance when training on CPUs
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
2025-09-28 11:13:28.986181: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-09-28 11:13:29.006240: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1759058009.027928  206418 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1759058009

In [ ]:
!CUDA_VISIBLE_DEVICES=0 python -m sglang.launch_server \
    --model-path ./qwen/ppo_medical_o1/checkpoint-32 \
    --port 28035 \
    --mem-fraction-static 0.8 \
    --dp 1 \
    --tp 1 > evaluation/sglang.log 2>&1 &

In [ ]:
!python evaluation/eval.py \
    --model_name ./qwen/ppo_medical_o1/checkpoint-32 \
    --eval_file evaluation/eval_data.json \
    --port 28035

Using local API server at port 28035


  0%|          | 0/9 [00:00<?, ?it/s]

Example:
Please answer the following multiple-choice question:
Which of the following is not true about glomerular capillaries')
A. The oncotic pressure of the fluid leaving the capillaries is less than that of fluid entering it
B. Glucose concentration in the capillaries is the same as that in glomerular filtrate
C. Constriction of afferent aeriole decreases the blood flow to the glomerulas
D. Hematocrit of the fluid leaving the capillaries is less than that of the fluid entering it


100%|██████████| 9/9 [04:15<00:00, 28.35s/it]


*checkpoint-32eval_data_api.json*
{
    "MedMCQA_validation": [
        0.4004303131723643,
        1674,
        4183,
        1675
    ],
    "MedQA_USLME_test": [
        0.4548311076197958,
        579,
        1273,
        572
    ],
    "PubMedQA_test": [
        0.643,
        643,
        1000,
        638
    ],
    "MMLU-Pro_Medical_test": [
        0.36612377850162864,
        562,
        1535,
        551
    ],
    "GPQA_Medical_test": [
        0.2923076923076923,
        114,
        390,
        113
    ]
}


In [ ]:
!pkill -f sglang
!pkill -f multiprocessing.spawn